In [3]:
import pandas as pd

In [4]:
test_df = pd.read_csv("../data/raw/test.csv")

In [6]:
test_df["building_id"].unique

<bound method Series.unique of 0              0
1              1
2              2
3              3
4              4
            ... 
41697595    1444
41697596    1445
41697597    1446
41697598    1447
41697599    1448
Name: building_id, Length: 41697600, dtype: int64>

In [7]:
test_grp = test_df.groupby("building_id")

In [11]:
test_grp.get_group(int("0"))

,row_id,building_id,meter,timestamp
0,0,0,0,2017-01-01 00:00:00
129,129,0,0,2017-01-01 01:00:00
258,258,0,0,2017-01-01 02:00:00
387,387,0,0,2017-01-01 03:00:00
516,516,0,0,2017-01-01 04:00:00
...,...,...,...,...
2259435,2259435,0,0,2018-12-31 19:00:00
2259564,2259564,0,0,2018-12-31 20:00:00
2259693,2259693,0,0,2018-12-31 21:00:00
2259822,2259822,0,0,2018-12-31 22:00:00
